In [1]:
from langchain_text_splitters import MarkdownHeaderTextSplitter

headers_to_split_on = [
    ("#", "title"),    
    ("##", "chapter"),    
    ("###", "section"),   
]

markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on)

In [3]:
from dotenv import load_dotenv
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings

load_dotenv()

embeddings = OpenAIEmbeddings(model = "text-embedding-3-large")
index_name = "inhouse-python-index"

vector_store = PineconeVectorStore(index_name=index_name, embedding=embeddings)

In [4]:
import os
input_dir = './output'

for filename in os.listdir(input_dir):
    if filename.endswith('.md'):
        md_path = os.path.join(input_dir, filename)
        with open(os.path.join(input_dir, filename), 'r', encoding='utf-8') as f:
            markdown_text = f.read()
        
        docs = markdown_splitter.split_text(markdown_text)
        for doc in docs:
            
            doc.metadata['source'] = filename.replace('.md','')
        vector_store.add_documents(docs)
            


In [5]:
retriver = vector_store.as_retriever()

In [6]:
retriver.invoke("본인 결혼식 경조휴가는 얼마인가요?")

[Document(id='322cd6f0-215d-4dac-af1d-7186c9bad066', metadata={'chapter': '3. 경조 휴가 및 지원', 'source': 'employee_benefits_and_welfare_faq', 'title': '직원 복리후생 및 복지 FAQ'}, page_content='**Q6: 경조 휴가는 어떤 경우에 사용할 수 있나요?**\n- A: 본인 결혼, 자녀 출생, 부모/배우자 사망 등 경조사 사유에 따라 휴가가 부여됩니다.  \n**Q7: 경조사 지원금은 어떻게 신청하나요?**\n- A: HR 시스템을 통해 신청할 수 있으며, 지원금은 지정된 계좌로 지급됩니다.  \n---'),
 Document(id='64d81f81-c5eb-4a5d-a8fd-4ef79c4e6095', metadata={'chapter': '제 3장 근로 시간 및 휴가', 'section': '제 6조 (근로 시간)', 'source': 'employee_handbook_and_hr_policy', 'title': '직원 핸드북 및 인사 정책'}, page_content='- 기본 근무 시간: 월요일~금요일, 오전 9시 ~ 오후 6시\n- 점심 시간: 오후 12시 ~ 오후 1시\n- 유연 근무제 가능 (사전 승인 필요)  \n제 7조 (지각, 결근)\n- 지각: 정해진 근무 시간 이후 도착할 경우\n- 결근: 사전 승인 없이 출근하지 않는 경우\n- 3회 지각 시 1회 결근으로 처리  \n제 8조 (연차 유급 휴가)\n- 연차 유급 휴가는 근속 연수에 따라 자동 지급한다.\n- 연차 휴가 지급 기준:  \n| 근속 연수 | 연차 휴가 일수 |\n|------------|----------------|\n| 1년 미만   | 월 1일         |\n| 1~3년      | 15일          |\n| 4~6년      | 17일          |\n| 7~9년      | 19일          |\n| 10년 이상  | 20일